# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [168]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl_df=nhl_df[nhl_df['year']==2018]
    population=pd.Series(list(cities['Population (2016 est.)[8]'].astype('float')),index=list(cities['Metropolitan area']))
    nhl_df['team']=nhl_df['team'].str.replace(r'\*$','',regex=True)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NHL']=cities['NHL'].str.replace(r'\[note\s\d+\]$','',regex=True)
    cities=cities.set_index('Metropolitan area')
    citiesNHL=cities['NHL']
    citiesNHL.replace(['','—'],np.nan,inplace=True)
    citiesNHL.dropna(inplace=True)
    citiesNHL_Aux=citiesNHL.copy()
    citiesNHL_Aux.drop(['Detroit','Toronto','Columbus'], inplace=True)
    l=citiesNHL_Aux.index.to_list()
    l1=[]
    l2=[]

    for area in l:
        for team in citiesNHL_Aux[area].split():
            l1.append(area)
            l2.append(team)

    for other in ['Detroit','Toronto','Columbus']:
            l2.append(citiesNHL[other])
            l1.append(other)

    citiesNHL_Aux2=pd.Series(l1,index=l2)

    for i in range(len(nhl_df)):
        if nhl_df.loc[i,'team'] in ['Detroit Red Wings', 'Toronto Maple Leafs','Columbus Blue Jackets']:
            nhl_df.loc[i,'team']=nhl_df.loc[i,'team'].split()[-2] + ' '+ nhl_df.loc[i,'team'].split()[-1]
        else:
            nhl_df.loc[i,'team']=nhl_df.loc[i,'team'].split()[-1]

    nhl_df.set_index('team',inplace=True)
    nhl_df=nhl_df[['W','L']]
    citiesNHL_Aux2.name='Metropolitan area'
    merge_df=nhl_df.join(pd.DataFrame(citiesNHL_Aux2), how='left')
    merge_df.drop('Division',inplace=True)
    merge_df['Ratio']=merge_df['W'].astype(float)/(merge_df['W'].astype(float)+merge_df['L'].astype(float))
    merge_df=merge_df.groupby('Metropolitan area')['Ratio'].mean().reset_index()
    merge_df['Population']=[population[area] for area in merge_df['Metropolitan area']]
    
    population_by_region = merge_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge_df['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [170]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nhl_df=nhl_df[nhl_df['year']==2018]
population=pd.Series(list(cities['Population (2016 est.)[8]'].astype('float')),index=list(cities['Metropolitan area']))
nhl_df['team']=nhl_df['team'].str.replace(r'\*$','',regex=True)
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL']=cities['NHL'].str.replace(r'\[note\s\d+\]$','',regex=True)
cities=cities.set_index('Metropolitan area')
citiesNHL=cities['NHL']
citiesNHL.replace(['','—'],np.nan,inplace=True)
citiesNHL.dropna(inplace=True)
citiesNHL_Aux=citiesNHL.copy()
citiesNHL_Aux.drop(['Detroit','Toronto','Columbus'], inplace=True)
l=citiesNHL_Aux.index.to_list()
l1=[]
l2=[]

for area in l:
    for team in citiesNHL_Aux[area].split():
        l1.append(area)
        l2.append(team)

for other in ['Detroit','Toronto','Columbus']:
        l2.append(citiesNHL[other])
        l1.append(other)

citiesNHL_Aux2=pd.Series(l1,index=l2)

for i in range(len(nhl_df)):
    if nhl_df.loc[i,'team'] in ['Detroit Red Wings', 'Toronto Maple Leafs','Columbus Blue Jackets']:
        nhl_df.loc[i,'team']=nhl_df.loc[i,'team'].split()[-2] + ' '+ nhl_df.loc[i,'team'].split()[-1]
    else:
        nhl_df.loc[i,'team']=nhl_df.loc[i,'team'].split()[-1]

nhl_df.set_index('team',inplace=True)
nhl_df=nhl_df[['W','L']]
citiesNHL_Aux2.name='Metropolitan area'
merge_df=nhl_df.join(pd.DataFrame(citiesNHL_Aux2), how='left')
merge_df.drop('Division',inplace=True)
merge_df['Ratio']=merge_df['W'].astype(float)/(merge_df['W'].astype(float)+merge_df['L'].astype(float))
merge_df=merge_df.groupby('Metropolitan area')['Ratio'].mean().reset_index()
merge_df['Population']=[population[area] for area in merge_df['Metropolitan area']]
